In [5]:
import json
import pandas as pd

from regressor_utils import *
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from fireTS.models import NARX

with open("data/feature_selections/xgb_reg_stage2.json") as f:
    features = json.loads(f.read())

sfm = features["SFM"]


def plot_results(mdl1, ytest, ypred):
    fig, axs = plt.subplots(1,3)
    fig.set_size_inches(16,6)

    axs[0].plot(ytest,label="Measured")
    axs[0].plot(ypred,label="Predicted")
    axs[0].legend()
    axs[0].set_title("Measure and prediction vs. time")

    axs[1].plot(ytest - ypred, label="Error")
    axs[1].legend()
    axs[1].set_title("Error ($y - \hat y$)")

    axs[2].scatter(ytest,ypred)
    axs[2].plot(ytest,ytest)
    axs[2].set_title("Measure vs. Prediction")
    plt.show()

    r2 = metrics.r2_score(np.nan_to_num(ypred),ytest)
    mse = metrics.mean_squared_error(np.nan_to_num(ypred),ytest)
    maxerr = abs(np.max(ytest-ypred))

    print("R2 score: ",r2)
    print("MSE: ", mse)
    print("Max error",maxerr)

    return (r2,mse,maxerr)

def get_scores(ytest, ypred):
    r2 = metrics.r2_score(np.nan_to_num(ypred),ytest)
    mse = metrics.mean_squared_error(np.nan_to_num(ypred),ytest)
    maxerr = abs(np.max(ytest-ypred))

    print("R2 score: ",r2)
    print("MSE: ", mse)
    print("Max error",maxerr)

    return (r2,mse,maxerr)


In [3]:
stage1_paths = [
    # STAGE 1
    "data/testes/teste_04_10.parquet",
    "data/testes/teste_18_10.parquet"
]

stage2_paths = [
    # STAGE 2
    "data/testes/teste_02_01_13h56min_14h14min.parquet",
    "data/testes/teste_02_01_17h33min_18h13min.parquet"
]

stage3_paths = [
    # STAGE 3
    "data/testes/teste_09_02.parquet"
]

In [ ]:
# Treino
df = pd.read_parquet(stage2_paths[0])
df2 = pd.read_parquet(stage2_paths[1])[:500]


x = df[sfm]
x2 = df2[sfm]

y = df["Consumption"]
y2 = df2["Consumption"]

delays = [2,3,4,5,6,7]
neurons = np.arange(30,110,5)

results = []

for delay in delays:
    for neuron in neurons:
        mdl1 = NARX(
            MLPRegressor(hidden_layer_sizes=(neuron),solver="adam",max_iter=2000),
            auto_order=delay,
            exog_order=x.shape[1]*[delay])
        mdl1.fit(x, y)

        xtest = x
        ytest = y.values
        yfrcast = mdl1.forecast(xtest[:delay],ytest[:delay],step = ytest.shape[0]-delay, X_future=xtest[delay:ytest.shape[0]-1])
        trainscores = get_scores(ytest[delay:],yfrcast)

        print("\n\nTest scores for features ", features)
        xtest = x2
        ytest = y2.values
        yfrcast = mdl1.forecast(xtest[:delay],ytest[:delay],step = ytest.shape[0]-delay, X_future=xtest[delay:ytest.shape[0]-1])
        testscores = get_scores( ytest[delay:],yfrcast)

        results.append((
            f"Delay = {delay}, N. Neurons = {neuron}",
            trainscores[0],
            trainscores[1],
            trainscores[2],
            testscores[0],
            testscores[1],
            testscores[2],
        ))

In [7]:
resultsdf = pd.DataFrame(results,columns=("Settings","train r2","train MSE","train max err","test r2","test MSE","test max err"))

resultsdf.to_excel("data/narx_grid_search.xls")

<ipython-input-7-654844151f87>:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  resultsdf.to_excel("narx_grid_search.xls")
